## lets start by doewload the libareis + random numbers


In [6]:
# התקנת כל הספריות הנדרשות ישירות לתוך ה-Kernel הנוכחי
%pip install seaborn matplotlib pandas numpy scikit-learn joblib nltk


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
import pandas as pd
import numpy as np
import joblib
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from pathlib import Path
import re
import nltk
from nltk.corpus import stopwords

# הגדרת Seed לשחזור תוצאות (דרישה מס' 3)
np.random.seed(42)

# הורדת מילון המילים (אם צריך)
nltk.download('stopwords', quiet=True)
STOP_WORDS = set(stopwords.words('english'))

# הגדרת נתיבים
PROJECT_DIR = Path(r"C:\Users\yuval\Desktop\לימודים\ML")
# וודאי שהשורה הזו בקוד שלך נראית ככה:
INPUT_FILE = PROJECT_DIR / "sensed_data.csv"
OUTPUT_FILE = PROJECT_DIR / "processed_data_separated.csv" # השם החדש
MODEL_DIR = PROJECT_DIR / "models"

if not MODEL_DIR.exists():
    MODEL_DIR.mkdir()

    

Loaded 714 articles.
Cleaning columns separately: ['title', 'trail_text', 'tags', 'body']...
Final count: 714
Sample cleaned Trail Text: yorgos poor things lanthimos reunites emma stone weird kidnapping thriller kim kardashian sarah paulson get right side law ryan murphys story


## 1. Pre-Processing (עיבוד מקדים)
בשלב זה ביצענו טיוב נתונים (Data Cleaning) כדי להמיר את הטקסט הגולמי לפורמט אחיד. הפעולות שבוצעו:
1. **טיפול בערכים חסרים וכפילויות:** הסרת כתבות ללא כותרת והסרת כפילויות לפי URL.
2. **Text Normalization:** המרת כל הטקסט לאותיות קטנות (Lowercase).
3. **Noise Reduction:** הסרת תווים מיוחדים, מספרים וסימני פיסוק באמצעות Regex.
4. **Stop Words Removal:** הסרת מילות קישור נפוצות (כגון "the", "is") באמצעות ספריית NLTK, כדי להקטין את המימד ולהתמקד במילים בעלות משמעות.

In [26]:
def clean_text_noise(text):
    """פונקציית ניקוי: Lowercase, Regex, Stopwords"""
    if not isinstance(text, str): return ""
    text = text.lower()
    text = re.sub(r'[^a-z\s]', '', text)
    words = text.split()
    filtered = [w for w in words if w not in STOP_WORDS and len(w) > 2]
    return " ".join(filtered)
  # תוספת חשובה: מילים ספציפיות שראינו שצריך להעיף ---
nltk.download('stopwords', quiet=True)
NLTK_STOP_WORDS = set(stopwords.words('english'))
CUSTOM_STOP_WORDS = {
    'says', 'said', 'says', 'telling', 'told',  # פעלים של דיבור (נפוץ בחדשות)
    'mr', 'ms', 'mrs', 'dr', 'of','is','in'                  # תארים
    'one', 'two', 'three',                     # מספרים במילים
    'would', 'could', 'should', 'may', 'might', # פעלים מודאליים חלשים
    'also', 'even', 'like', 'get', 'go',       # מילים כלליות מדי
    'new', 'news', 'report', 'time', 'year',   # מילים שנפוצות בכל העיתון
    'people', 'us', 'uk', 'world'              # מקומות/אנשים כלליים
}

# 3. איחוד הרשימות לרשימה אחת חזקה
FULL_STOP_WORDS = NLTK_STOP_WORDS.union(CUSTOM_STOP_WORDS)

# טעינת הנתונים
if INPUT_FILE.exists():
    df = pd.read_csv(INPUT_FILE)
    print(f"Loaded {len(df)} articles.")

    # 1. ניקוי שורות בסיסי (כפילויות וחסרים)
    df = df.dropna(subset=['title'])
    df = df.drop_duplicates(subset=['url'])

    # 2. ניקוי שם הכותב
    df['author'] = df['author'].fillna('unknown').astype(str)
    df['author'] = df['author'].apply(lambda x: x.lower().replace('by ', '').strip())

    # 3. ניקוי עמודות הטקסט (בנפרד!)
    text_columns = ['title', 'trail_text', 'tags', 'body']
    print(f"Cleaning columns separately: {text_columns}...")

    for col in text_columns:
        # מילוי ערכים חסרים
        if col in df.columns:
            df[col] = df[col].fillna('')
            df[col] = df[col].apply(clean_text_noise)

    # 4. סינון שורות ללא תקציר (Trail Text)
    df = df[df['trail_text'].str.len() > 2]

    # הצגת דוגמה לניקוי
    print(f"Final count: {len(df)}")
    print("Sample cleaned Trail Text:", df['trail_text'].iloc[0])
    # הצגת השוואה בין המקור לניקוי עבור 5 הכתבות הראשונות
    print(df[['trail_text', 'clean_trail_text']].head())
    
    # שמירה זמנית (כדי שהשלב הבא יוכל להשתמש בזה)
    df.to_csv(OUTPUT_FILE, index=False)
else:
    print(f"Error: {INPUT_FILE} not found.")

Loaded 714 articles.
Cleaning columns separately: ['title', 'trail_text', 'tags', 'body']...
Final count: 714
Sample cleaned Trail Text: yorgos poor things lanthimos reunites emma stone weird kidnapping thriller kim kardashian sarah paulson get right side law ryan murphys story


KeyError: "['clean_trail_text'] not in index"

## 2. Feature Extraction (חילוץ מאפיינים)
בשלב זה נמיר את המידע הטקסטואלי למאפיינים מספריים בשלוש שיטות:
1.  **Trail Text (תקציר):** שימוש ב-TF-IDF כדי לזהות מילות מפתח בתקציר הכתבה (1,000 המילים המובילות).
2.  **Tags (תגיות):** שימוש ב-TF-IDF על התגיות המקצועיות (500 התגיות המובילות).
3.  **Authors (כותבים):** זיהוי כותבים "מומחים" (Specialists) וייצוגם כמאפיינים בינאריים.

In [24]:
def analyze_top_features(df, vectorizer, tfidf_matrix, feature_type_name):
    """מדפיסה את המילים/תגיות החזקות ביותר בכל קטגוריה"""
    print(f"\n--- Top Significant {feature_type_name} per Category ---")
    unique_labels = df['label'].unique()
    feature_names = np.array(vectorizer.get_feature_names_out())

    for label in unique_labels:
        indices = df.index[df['label'] == label].tolist()
        if not indices: continue

        class_matrix = tfidf_matrix[indices]
        mean_scores = np.array(class_matrix.mean(axis=0)).flatten()
        top_indices = mean_scores.argsort()[-10:][::-1]

        print(f"Category: {label.upper()}")
        results = [f"{feature_names[i]}" for i in top_indices]
        print(", ".join(results))

def get_specialist_authors(df, min_articles=3):
    """מחזירה רשימת כותבים שכתבו לפחות 3 כתבות ורק בנושא אחד"""
    print(f"\nAnalyzing Author Specialization...")
    df_clean = df[~df['author'].isin(['unknown', 'unknown author', 'guardian staff'])]
    author_matrix = pd.crosstab(df_clean['author'], df_clean['label'])
    active_authors = author_matrix[author_matrix.sum(axis=1) >= min_articles]
    is_specialist = (active_authors > 0).sum(axis=1) == 1
    specialist_authors = active_authors[is_specialist].index.tolist()
    print(f"Found {len(specialist_authors)} specialist authors.")
    return specialist_authors

In [27]:
# --- A. Trail Text Features ---
print("\n1. Extracting Trail-Text Features (Top 1000)...")

# התיקון: הוספנו stop_words='english' כדי להעיף את the, and, of
trail_vectorizer = TfidfVectorizer(max_features=1000, stop_words='english')

# אם את רוצה להוסיף גם את המילים הספציפיות שלך (כמו says), אפשר להעביר רשימה (אופציונלי)
# אבל stop_words='english' יעשה את רוב העבודה הקשה
X_trail = trail_vectorizer.fit_transform(df['trail_text'])

# הצגת הניתוח (כדי לראות שהמילים הגיוניות עכשיו)
analyze_top_features(df, trail_vectorizer, X_trail, "Trail-Words")

trail_features = pd.DataFrame(
    X_trail.toarray(), 
    columns=[f"trail_{w}" for w in trail_vectorizer.get_feature_names_out()]
)
joblib.dump(trail_vectorizer, MODEL_DIR / "tfidf_trail.pkl")

# --- B. Tag Features ---
print("\n2. Extracting Tag Features (Top 500)...")
# גם כאן כדאי להוסיף ליתר ביטחון, למרות שבתגיות זה פחות קריטי
tags_vectorizer = TfidfVectorizer(max_features=500, stop_words='english')
X_tags = tags_vectorizer.fit_transform(df['tags'])

# הצגת הניתוח
analyze_top_features(df, tags_vectorizer, X_tags, "Tags")

tags_features = pd.DataFrame(
    X_tags.toarray(), 
    columns=[f"tag_{w}" for w in tags_vectorizer.get_feature_names_out()]
)
joblib.dump(tags_vectorizer, MODEL_DIR / "tfidf_tags.pkl")


1. Extracting Trail-Text Features (Top 1000)...

--- Top Significant Trail-Words per Category ---
Category: CULTURE
latenight, hosts, trumps, new, discuss, discussed, weeks, shutdown, star, years
Category: NEWS
plus, read, years, data, days, need, pieces, climate, weekend, favourite
Category: OPINION
says, guardian, columnist, editorial, government, new, asks, writes, people, president
Category: SPORT
england, ashes, australia, test, win, blacks, englands, team, season, victory

2. Extracting Tag Features (Top 500)...

--- Top Significant Tags per Category ---
Category: CULTURE
culture, television, comedy, saturday, features, news, arts, music, article, radio
Category: NEWS
news, long, weather, read, weekly, guardian, main, section, features, crunch
Category: OPINION
opinion, comment, news, politics, matters, journal, article, trump, labour, donald
Category: SPORT
sport, cricket, team, rugby, australia, union, ashes, news, sports, newsletter


['C:\\Users\\yuval\\Desktop\\לימודים\\ML\\models\\tfidf_tags.pkl']

In [15]:
if INPUT_FILE.exists():
    # --- C: Author Features ---
    print("\n3. Extracting Specialist Author Features...")
    selected_authors = get_specialist_authors(df, min_articles=3)
    joblib.dump(selected_authors, MODEL_DIR / "authors_list.pkl")
    
    author_features = pd.DataFrame()
    for author in selected_authors:
        col_name = f"auth_{author.replace(' ', '_')}"
        author_features[col_name] = df['author'].apply(lambda x: 1 if x == author else 0)

    # --- D: Combine and Save ---
    print("\n4. Combining all features...")
    df.reset_index(drop=True, inplace=True)
    trail_features.reset_index(drop=True, inplace=True)
    tags_features.reset_index(drop=True, inplace=True)
    author_features.reset_index(drop=True, inplace=True)

    final_dataset = pd.concat([trail_features, tags_features, author_features], axis=1)
    final_dataset['label'] = df['label']
    
    final_dataset.to_csv(OUTPUT_FILE, index=False)
    print(f"\nSuccess! Final dataset saved to: {OUTPUT_FILE}")
    print(f"Dataset Dimensions: {final_dataset.shape}")
    print(f" - Trail Text features: {trail_features.shape[1]}")
    print(f" - Tag features: {tags_features.shape[1]}")
    print(f" - Author features: {author_features.shape[1]}")


3. Extracting Specialist Author Features...

Analyzing Author Specialization...
Found 55 specialist authors.

4. Combining all features...

Success! Final dataset saved to: C:\Users\yuval\Desktop\לימודים\ML\dataset_features_final.csv
Dataset Dimensions: (714, 1556)
 - Trail Text features: 1000
 - Tag features: 500
 - Author features: 55
